In [1]:
%matplotlib inline

In [2]:
from glob import glob

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [34]:
from matplotlib import style
style.use('seaborn-notebook')

In [5]:
# l = !ls ../data/hourly_data_nc/

In [25]:
def make_title(station):
    name = geodata.ix[int(station)]['NOM_USUEL']
    lon = geodata.ix[int(station)]['LON_DG']
    lat = geodata.ix[int(station)]['LAT_DG']
    alt = geodata.ix[int(station)]['ALTI']
    title = "{}: [{:4.2f}E / {:4.2f}S], {:4.2f} m.".format(name, lon, lat, alt)
    return lat, lon, title

In [7]:
# for f in l: 
#     print("processing {}".format(f))
#     data = pd.read_csv('../data/hourly_data_nc/{}'.format(f), sep=';', \
#                        header=None, parse_dates = {'date': [1, 2, 3, 4]}, \
#                        index_col='date')
#     data.columns = ['station', 'rain', 'code']
#     data.to_csv('../data/{}.csv'.format(f[:-4]))
#     del(data)

In [8]:
geodata = pd.read_csv('../data/geo.csv', index_col=0, sep=';')

In [9]:
geodata.head()

,NOM_USUEL,LON_DG,LAT_DG,LAMBX_100,LAMBY_100,ALTI
NUM_POSTE,,,,,,
98818001,NOUMEA,166.452833,-22.276000,4466.70,2140.19,70
98814001,OUANAHAM,167.240500,-20.777000,5291.64,3795.25,30
98822001,POINDIMIE,165.328000,-20.932500,3301.03,3626.73,14
98812001,KOUMAC,164.284167,-20.558667,2210.79,4032.26,25
98832101,GORO_ANCIENNE_PEPINIERE,166.967500,-22.269167,4997.17,2145.35,298


In [10]:
lfiles = glob('../data/98*.csv')

In [11]:
w = 6
h = 4
res = 80

In [35]:
for f in lfiles: 
    station = f.split('/')[-1][:-4]
    lat, lon, title = make_title(station)
    data = pd.read_csv(f, parse_dates=True, engine='python', index_col='date')
    datan = data.copy()
    datan.replace({'rain': {0: np.nan}}, inplace=True)
    datan.dropna(inplace=True)
    # rainfall frequency per hour (count of non-zero rainfall)
    freq_cycle = datan.groupby(datan.index.hour)[['rain']].count()
    # rainfall average per hour
    mean_cycle = data.groupby(data.index.hour)[['rain']].mean()
    # plots
    f, ax1 = plt.subplots(figsize=(w,h))
    f.subplots_adjust(right=0.85)
    ax2 = ax1.twinx()
    ax1.plot(freq_cycle, 'r', label='frequency', lw=3)
    ax2.plot(mean_cycle, 'b', label='average', lw=3)
    ax1.legend()
    ax2.legend(loc=4)
    ax1.set_ylabel('frequency')
    ax2.set_ylabel('average')
    ax1.set_xlim(0,23)
    ax1.set_xlabel('hour')
    ax1.set_title(title)
    f.savefig('./{}.png'.format(station), dpi=res)
    plt.close(f)

### maps

In [27]:
import folium

In [28]:
from map_notebook import * # local functions

In [29]:
map = folium.Map(location=[-21, 166.4580], zoom_start=8, tiles='Mapbox', API_key='nicolasf.ic0ebom5')

In [30]:
for f in lfiles: 
    station = f.split('/')[-1][:-4]
    fname = '{}.png'.format(station)
    lat, lon, title = make_title(station)
    textpop = """<div align='center'>
    <img src='{}' alt={} width='{}' height='{}'> </div>""".format(fname, title, w*res, h*res)
    map.circle_marker(location=[lat, lon], popup=textpop, fill_color='#FF0000', line_color=None, radius=2500)

In [31]:
inline_map(map)

In [32]:
embed_map(map)